In [157]:
from utils.postgre_database_connection import connect_to_local_database
from tqdm import tqdm
import pandas as pd
import psycopg2
import glob
import os
import re

In [158]:
n_files = 2

In [159]:
get_all_files_path_product = glob.glob('../assets/datasets/*.csv')[:n_files]
get_all_files_path_product

['../assets/datasets\\Air Conditioners.csv',
 '../assets/datasets\\All Appliances.csv']

In [87]:
for value in get_all_files_path_product:
    get_file_name = value.split(os.path.sep)[-1][:-4]
    print(get_file_name)

Air Conditioners
All Appliances


In [136]:
df = pd.read_csv(get_all_files_path_product[0])

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            720 non-null    object
 1   main_category   720 non-null    object
 2   sub_category    720 non-null    object
 3   image           720 non-null    object
 4   link            720 non-null    object
 5   ratings         433 non-null    object
 6   no_of_ratings   433 non-null    object
 7   discount_price  457 non-null    object
 8   actual_price    500 non-null    object
dtypes: object(9)
memory usage: 50.8+ KB


In [26]:
df[:5]

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990"
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790"


In [28]:
df.columns

Index(['name', 'main_category', 'sub_category', 'image', 'link', 'ratings',
       'no_of_ratings', 'discount_price', 'actual_price'],
      dtype='object')

In [160]:
try: 
    connection = connect_to_local_database()
    cursor = connection.cursor()
    print("Successfully connected to the database.")
    
    # cek konfigurasi apakah sudah sesuai atau belum, kalo ragu bisa di uncomment
    # cursor.close()
    # connection.close() 
except (Exception, psycopg2.DatabaseError) as error:
    print(f'Error: {error}')

Successfully connected to the database.


In [161]:
CREATE_TABLE_QUERY = """
--sql
CREATE TABLE amazon_products.{} ()
;
"""

ADD_COLUMN_QUERY = """
--sql
ALTER TABLE amazon_products.{}
    ADD COLUMN {}
;
"""

INSERT_VALUES_QUERY = """
--sql
INSERT INTO amazon_products.{} ({})
VALUES ({})
;
"""

for file_path in get_all_files_path_product:
    table_name = file_path.split(os.path.sep)[-1][:-4]
    table_name = re.sub(r"-|\s+", "_", table_name).lower()
    
    if table_name != 'amazon_products':
        df = pd.read_csv(file_path)
        if df.shape[0] != 0:
            cursor.execute(CREATE_TABLE_QUERY.format(table_name))

            column_names = []
            for column in df.columns:
                column_name = re.sub(r"\s+", "_", column).lower()  # Replace spaces in column names
                cursor.execute(ADD_COLUMN_QUERY.format(table_name, column_name + " TEXT"))
                column_names.append(column_name)
            
            # Construct the placeholders for the values
            placeholders = ", ".join(["%s"] * len(column_names))
            columns_formatted = ", ".join(column_names)

            for value in tqdm(df.values, desc=f"Inserting Values into {table_name}"):
                cursor.execute(INSERT_VALUES_QUERY.format(table_name, columns_formatted, placeholders), tuple(value))

            print(f'Tabel "{table_name}" berhasil dibuat')
            print(f'Total baris "{df.shape[0]}", Total kolom "{df.shape[1]}"')
            print("=".center(60, "="))

connection.commit()
print('Proses Pemindahan Berhasil')
cursor.close()
connection.close() 

Inserting Values into air_conditioners: 100%|██████████| 720/720 [00:00<00:00, 2946.06it/s]


Tabel "air_conditioners" berhasil dibuat
Total baris "720", Total kolom "9"


Inserting Values into all_appliances: 100%|██████████| 9576/9576 [00:03<00:00, 2581.05it/s]

Tabel "all_appliances" berhasil dibuat
Total baris "9576", Total kolom "9"
Proses Pemindahan Berhasil
